In [49]:
# Import necessary libraries
import pandas as pd         # For data manipulation
import re                   # For regex operations
from datetime import datetime  # For date handling
import warnings             # To suppress unnecessary warnings
from glob import glob       # For dynamic file handling
import os                   # For operating system interactions

# Suppress RuntimeWarnings globally
warnings.filterwarnings("ignore", category=RuntimeWarning)

# Print a message to ensure the environment is set up
print("Environment is set up. Libraries are imported.")


Environment is set up. Libraries are imported.


In [50]:
# --- File Handling ---
def get_files_in_directory(directory, pattern="*.xlsx"):
    """
    Fetch all files matching a specific pattern from a directory.

    Args:
    - directory (str): Path to the directory.
    - pattern (str): Pattern to match (e.g., "*.xlsx").

    Returns:
    - list: List of file paths.
    """
    return glob(os.path.join(directory, pattern))

In [51]:
# --- Input Validation ---
def validate_inputs(fund_name, start_date, end_date):
    """
    Validates user inputs for fund name and date range.

    Args:
    - fund_name (str): The name of the fund provided by the user.
    - start_date (str): The start date in YYYY-MM-DD format.
    - end_date (str): The end date in YYYY-MM-DD format.

    Returns:
    - bool: True if inputs are valid, False otherwise.
    """
    try:
        # Ensure the dates are valid and in the correct order
        start_date = datetime.strptime(start_date, "%Y-%m-%d")
        end_date = datetime.strptime(end_date, "%Y-%m-%d")
        if start_date > end_date:
            print("Error: Start date must be earlier than or equal to the end date.")
            return False
    except ValueError:
        print("Error: Invalid date format. Use YYYY-MM-DD.")
        return False

    if not fund_name.strip():
        print("Error: Fund name cannot be empty.")
        return False

    return True


In [52]:
# --- Data Loading ---
def load_monthly_data(file_paths, start_date, end_date):
    """
    Load and combine monthly data within the specified date range.

    Args:
    - file_paths (list): List of file paths.
    - start_date (str): Start date in YYYY-MM-DD format.
    - end_date (str): End date in YYYY-MM-DD format.

    Returns:
    - pd.DataFrame: Combined data from all valid files.
    """
    dataframes = []
    start_date = datetime.strptime(start_date, "%Y-%m-%d")
    end_date = datetime.strptime(end_date, "%Y-%m-%d")

    for file in file_paths:
        try:
            match = re.search(r"Monthly Portfolio (\w+ \d{4})", file)
            if match:
                file_date_str = match.group(1)
                file_date = datetime.strptime(file_date_str, "%B %Y")
            else:
                print(f"Skipping file (invalid date format): {file}")
                continue
        except ValueError:
            print(f"Skipping file (invalid date format): {file}")
            continue

        if start_date <= file_date <= end_date:
            print(f"Loading file: {file}")
            df = pd.read_excel(file, skiprows=4)
            df['Month'] = file_date.strftime('%B %Y')
            dataframes.append(df)

    if not dataframes:
        raise ValueError("No files found in the specified date range.")

    return pd.concat(dataframes, ignore_index=True)

In [53]:
# --- Data Processing ---
def process_data(data):
    """
    Clean and process the loaded data for further analysis.

    Args:
    - data (pd.DataFrame): Raw data loaded from Excel files.

    Returns:
    - pd.DataFrame: Processed data ready for analysis.
    """
    columns_to_rename = {
        'Unnamed: 2': 'Instrument Name',
        'Unnamed: 4': 'ISIN',
        'Unnamed: 5': 'Rating / Industry',
        'Unnamed: 6': 'Quantity',
        'Unnamed: 7': 'Market Value (Rs. in Lakhs)',
        'Unnamed: 8': '% to NAV',
    }
    data.rename(columns=columns_to_rename, inplace=True)
    data.dropna(subset=['Instrument Name'], inplace=True)
    data = data[~data['Instrument Name'].str.contains('a) Listed/awaiting listing', na=False, regex=False)]
    data.reset_index(drop=True, inplace=True)

    for column in ['Quantity', 'Market Value (Rs. in Lakhs)']:
        data.loc[:, column] = pd.to_numeric(data[column], errors='coerce')

    return data


In [54]:
def analyze_fund_changes(processed_data, fund_name=None):
    """
    Analyze changes in fund allocations with absolute and percentage changes.

    Args:
    - processed_data (pd.DataFrame): Cleaned and processed data.
    - fund_name (str, optional): Specific fund name to filter by.

    Returns:
    - pd.DataFrame: Final analyzed data with changes across months.
    """
    if fund_name:
        processed_data = processed_data[
            processed_data['Instrument Name'].str.contains(fund_name, case=False, na=False)
        ]

    grouped_data = processed_data.groupby(['Instrument Name', 'ISIN', 'Month']).agg({
        'Quantity': 'sum',
        'Market Value (Rs. in Lakhs)': 'sum'
    }).reset_index()

    pivot_quantity = grouped_data.pivot(index=['Instrument Name', 'ISIN'], columns='Month', values='Quantity')
    pivot_market_value = grouped_data.pivot(index=['Instrument Name', 'ISIN'], columns='Month', values='Market Value (Rs. in Lakhs)')

    pivot_quantity['Quantity Change'] = pivot_quantity.iloc[:, -1] - pivot_quantity.iloc[:, 0]
    
    # Convert 'Market Value Change' to numeric, handling errors by coercing invalid values to NaN
    pivot_market_value['Market Value Change (Rs. in Lakhs)'] = pd.to_numeric(
        pivot_market_value.iloc[:, -1] - pivot_market_value.iloc[:, 0], errors='coerce'
    )

    pivot_market_value['Market Value Change (%)'] = (
        (pivot_market_value.iloc[:, -1] - pivot_market_value.iloc[:, 0]) / pivot_market_value.iloc[:, 0]
    ) * 100

    final_output = pd.merge(
        pivot_quantity.reset_index(),
        pivot_market_value.reset_index(),
        on=['Instrument Name', 'ISIN']
    )

    # Sort the data based on 'Market Value Change (Rs. in Lakhs)' after ensuring it's numeric
    sorted_data = final_output.sort_values(by='Market Value Change (Rs. in Lakhs)', ascending=False)

    return sorted_data


In [55]:
# --- Save and Display Results ---
def save_and_display_results(sorted_data, output_path='Mutual_Fund_Analysis.xlsx'):
    """
    Save the results to an Excel file and display the top changes.

    Args:
    - sorted_data (pd.DataFrame): Analyzed data sorted by market value changes.
    - output_path (str): Path to save the output Excel file.
    """
    sorted_data.to_excel(output_path, index=False)
    print("\nTop Changes in Portfolio Allocation:")
    print(sorted_data.head(10))
    print(f"\nResults saved to {output_path}")

In [56]:
# --- Main Function ---
if __name__ == "__main__":
    fund_name = input("Enter Fund Name (e.g., HDFC Bank Limited): ").strip()
    start_date = input("Enter Start Date (YYYY-MM-DD): ").strip()
    end_date = input("Enter End Date (YYYY-MM-DD): ").strip()

    directory = input("Enter the directory path containing mutual fund files: ").strip()
    file_paths = get_files_in_directory(directory)

    if validate_inputs(fund_name, start_date, end_date):
        try:
            all_data = load_monthly_data(file_paths, start_date, end_date)
            processed_data = process_data(all_data)
            analyzed_data = analyze_fund_changes(processed_data, fund_name=fund_name)
            save_and_display_results(analyzed_data)
        except ValueError as e:
            print(e)
        except Exception as e:
            print(f"An error occurred: {e}")
    else:
        print("Invalid inputs. Exiting program.")

Enter Fund Name (e.g., HDFC Bank Limited):  Infosys Limited
Enter Start Date (YYYY-MM-DD):  2024-09-01
Enter End Date (YYYY-MM-DD):  2024-11-30
Enter the directory path containing mutual fund files:  /kaggle/input/zerodha


Loading file: /kaggle/input/zerodha/ZN250 - Monthly Portfolio September 2024.xlsx
Loading file: /kaggle/input/zerodha/ZN250 - Monthly Portfolio November 2024.xlsx

Top Changes in Portfolio Allocation:
Month  Instrument Name           ISIN November 2024_x September 2024_x  \
0      Infosys Limited  IT - Software         1758.46          1487.33   

Month Quantity Change November 2024_y September 2024_y  \
0             -271.13        0.024903         0.023724   

Month  Market Value Change (Rs. in Lakhs) Market Value Change (%)  
0                               -0.001179             -104.734664  

Results saved to Mutual_Fund_Analysis.xlsx
